In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
# Load the dataset
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Pre-process the data
# Drop the irrelevant columns

data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Encode Categorical variables

label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# One hot encode 'Geography' column

from sklearn.preprocessing import OneHotEncoder

one_hot_encoder_geo=OneHotEncoder()
geo_encoder=one_hot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder


<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [6]:
one_hot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [8]:
geo_encoder_df=pd.DataFrame(geo_encoder.toarray(),columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [9]:
## Combine one hot encoder columns with the original data

data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

NameError: name 'geo_encoded_df' is not defined

In [10]:
# Save the encoders and scaler

with open('label_encoder_gender.pkl','wb') as file:   # wb is write byte mode
    pickle.dump(label_encoder_gender,file)

with open('one_hot_encoder_geo.pkl','wb') as file:   # wb is write byte mode
    pickle.dump(one_hot_encoder_geo,file)    

In [ ]:
# Divide the data set into independent and dependent features

X=data.drop('Exited', axis=1)
y=data['Exited']

# Split the data in training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=8)

# Scale the features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [12]:
# Save the encoders and scaler

with open('scaler.pkl','wb') as file:   # wb is write byte mode
    pickle.dump(scaler,file)

ANN Implementation

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [14]:
 # Build our ANN model

model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #HL1 connected with input layer
    Dense(32,activation='relu'), #HL2
    Dense(1,activation='sigmoid') # output layer
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [54]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.001)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [55]:
# Compile the model

model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy']) # or we can give optimizer='adam'

In [56]:
# Set-up Tensorboard

from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [59]:
# Set-up Early Stopping 

early_stopping_callback=EarlyStopping(monitor='val_loss',patience=15,restore_best_weights=True)

In [60]:
# Train the model

history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.2816 - accuracy: 0.8839 - val_loss: 0.3554 - val_accuracy: 0.8605
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.2790 - accuracy: 0.8849 - val_loss: 0.3585 - val_accuracy: 0.8640
Epoch 3/100
250/250 [==============================] - 2s 7ms/step - loss: 0.2772 - accuracy: 0.8839 - val_loss: 0.3601 - val_accuracy: 0.8630
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.2753 - accuracy: 0.8854 - val_loss: 0.3638 - val_accuracy: 0.8620
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.2740 - accuracy: 0.8864 - val_loss: 0.3648 - val_accuracy: 0.8600
Epoch 6/100
250/250 [==============================] - 1s 6ms/step - loss: 0.2724 - accuracy: 0.8873 - val_loss: 0.3669 - val_accuracy: 0.8620
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.2703 - accuracy: 0.8875 - val_loss: 0.3673 - val_accuracy: 0.8575

In [61]:
model.save('model.h5')

In [64]:
# Load Tensorflow Extension

#%load_ext tensorboard
%reload_ext tensorboard

In [65]:
# Launch Tensorboard

#%tensorboard --logdir logs/fit
#%tensorboard --logdir logs/fit20250626-195445
%tensorboard --logdir logs/fit20250626-202240

Reusing TensorBoard on port 6010 (pid 6624), started 0:01:31 ago. (Use '!kill 6624' to kill it.)